In [14]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 1.8 MB/s eta 0:00:00.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import folium
from folium.plugins import MarkerCluster

# Load dataset
df = pd.read_csv("atm_pune_dataset.csv")

# Create shutdown label
df["Shutdown_Label"] = ((df["Monthly_Transactions"] < 200) | (df["Last_Used_Days_Ago"] > 60)).astype(int)

# Feature engineering: One-hot encoding
df_model = pd.get_dummies(df.copy(), columns=["Location", "Area_Type"], drop_first=True)

# Define features and target
X = df_model.drop(columns=["ATM_ID", "City", "ATM_Status", "Shutdown_Label"])
y = df_model["Shutdown_Label"]

# Train-test split
X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(
    X, y, df, test_size=0.2, random_state=42, stratify=y
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Tune KNN with GridSearchCV
param_grid = {
    'n_neighbors': list(range(3, 21, 2)),
    'weights': ['uniform', 'distance']
}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train_scaled, y_train)
best_knn = grid.best_estimator_

# Predictions
y_pred = best_knn.predict(X_test_scaled)
df_test["Predicted_Shutdown"] = y_pred
shutdown_df = df_test[df_test["Predicted_Shutdown"] == 1]

# Save results
shutdown_df.to_csv("predicted_atms_to_shutdown.csv", index=False)
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("knn_classification_report.csv")

# Create Folium map of shutdown ATMs
m = folium.Map(location=[18.5204, 73.8567], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m)

for _, row in shutdown_df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=f"<b>{row['ATM_ID']}</b><br>{row['Location']}<br>Txns: {row['Monthly_Transactions']}",
        icon=folium.Icon(color='red', icon='times-circle', prefix='fa')
    ).add_to(marker_cluster)

m.save("atm_shutdown_map.html")

print("Done. Files created:")
print("- predicted_atms_to_shutdown.csv")
print("- knn_classification_report.csv")
print("- atm_shutdown_map.html")


Done. Files created:
- predicted_atms_to_shutdown.csv
- knn_classification_report.csv
- atm_shutdown_map.html
